Testing the implementaion of All positive values using 12 Qubits - 4096 data points. The weights are obtained from pre training using a simulator and tested in the Cat Qubits.

The experimental code for developing Amplitude embedded circuits using Variational Quantum State Preparation (VQSP). The VQSP algorithm uses PQC to create training circuits which learns the parameters to create the corresponding amplitude embedded states.

In [1]:
# !pip install pennylane nftopt

In [2]:
# !pip install git+https://github.com/QuantumETS/pennylane-alice-bob.git@main

We specify the number of qubits needed to map the data, for n qubits we can amplitude embedded 2^n data. The depth represents the ansatz repitition depth.

In [3]:
#defining parameters
n_qubits = 12
depth = 4

In [4]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod

We use the Logical Cat qubit version as the physical cat qubit version does not support the gate configurations that have been used in the circuit. Repo Link - <a> https://github.com/QuantumETS/pennylane-alice-bob/tree/main </a>

In [5]:
# dev = qml.device("alicebob.qubit", alice_backend="EMU:40Q:PHYSICAL_CATS", wires=n_qubits, average_nb_photons=4,kappa_1=1e2, kappa_2=1e4, shots=1000)
dev = qml.device("alicebob.qubit", alice_backend="EMU:15Q:LOGICAL_EARLY", wires=n_qubits)
dev

Using alice & bob EMU:15Q:LOGICAL_EARLY backend...


<RemoteDevice device (wires=12, shots=1024) at 0x7fedddcdb7f0>

A single Ansatz layer used for training contains set of trainiable RY gates for all the qubits followed by cyclic CZ arrangement.

In [6]:
def ansatz_layer(layer_weights, depth, n_qubits):
    for dep in range(depth):
        for wire in range(n_qubits):
            qml.RY(layer_weights[wire+(n_qubits*dep)], wires=wire)

        for i in range(n_qubits-1):
            qml.CZ(wires=[i,i+1])
        qml.CZ(wires=[n_qubits-1,0])

We use the Fidelity as the accuracy metric in this case. The Fidelity estimate provides the similarity between the generated state and the required amplitude embedding. It returs 1 if there are similar.

In [7]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [8]:
@qml.qnode(dev)
def circuit(weights):
  # since depth 4 was used
  ansatz_layer(weights,depth=depth, n_qubits=n_qubits)
  # qml.state() applies Ua to Ini State, which gives Appro_a
  # return qml.state()
  return qml.probs()

In [9]:
def variational_classifier(weights, x):
    # weights are thetas
    return np.real(circuit(weights))

Loading the data which have been used to get the weights by previous training

In [10]:
value = np.loadtxt('12_qubit_data.out')
print(value)

[0.00975932 0.02195996 0.00954274 ... 0.01582369 0.02101887 0.01189177]


Loading the weights to prepare the amplitude embedding of the above data

In [11]:
weights = np.loadtxt('12_qubit_weights.out')
print(weights)

[ 1.34730403e+01  6.08781112e+00  2.23222438e+00  9.41198146e+00
  7.19025136e+00  6.27874028e+00  3.27355972e+00  1.59422817e-02
  6.16736248e+00  3.13499254e+00  1.02795449e+01  3.14328155e+00
  3.13860280e+00  3.11840574e+00  1.17668482e-02  9.42731269e+00
  3.22188528e+00  6.28057554e+00  4.31305869e+00  3.14920044e+00
  1.12584161e+01  3.11340870e+00  6.77459811e+00  1.25607068e+01
  2.46688751e+00  4.56249561e+00  5.60919706e+00  4.69490231e+00
  2.40140690e+00  7.85524717e+00  9.18206385e+00  1.57091538e+00
  3.63954967e-01  4.70750144e+00  2.93105472e+00  7.85840491e+00
  9.42815699e+00 -2.74568408e-02  9.42519845e+00  3.14230608e+00
  3.13708694e+00  9.42858711e+00  1.25697236e+01  9.41732257e+00
  1.25687459e+01  9.39739819e+00  1.25707670e+01  6.28472098e+00]


We compite the fidelity of the generated states using VQSP and the original amplitude embedding needed.

The Cat qubit library does not support gathering state vector from circuit, so we get the probabilities of the result.

Since (Amplitude of state) ^ 2 = Probability,
we compute the squre root of Probability before finding the fidelity of the states.

In [12]:
predictions=variational_classifier(weights,value)
print(predictions)
# Amp (qml.state) ^ 2 = prob (qml.prob)
acc = accuracy(value, np.sqrt(predictions))
# print(predictions.numpy(), x.numpy())
print(acc)

[0.         0.00195312 0.00097656 ... 0.         0.         0.00097656]
0.18937636074588654
